In [30]:
import inspect
from copy import deepcopy, copy
from inspect import Parameter, Signature
from types import FunctionType
from typing import *

from mousse import Accessor, validator

In [56]:
class Accessor:
    def __init__(self, key: str, default: Any = None, validators: List[str] = None):
        self.key = key
        self.default = default
        self.validators = validators or []
        self.storage = {}
    
    def __get__(self, obj: "Dataclass", *args, **kwargs):
        if obj not in self.storage:
            self.storage[obj] = deepcopy(self.default)
        return self.storage.get(obj)

    def __set__(self, obj: "Dataclass", val: Any):
        for validator_name in self.validators:
            validator = getattr(obj, validator_name)
            assert validator(val), f"Validation failed at: {validator_name}"

        self.storage[obj] = val
        
        
class Validator:
    def __init__(self, field: str, func: Callable):
        self.field = field
        self.func = func
        
        
def validator(field: str, func: Callable = None):
    def decorator(func: Callable):
        return Validator(field, func)
    
    if func is not None:
        return decorator(func)
    
    return decorator



class DataMetaclass(type):
    def __new__(cls, name: str, bases: Tuple[type, ...], data: Dict[str, Any], accessor: Type[Accessor] = Accessor):
        validators = {}
        for key, val in data.items():
            if type(val) is Validator:
                if val.field not in validators:
                    validators[val.field] = []
                
                validators[val.field].append(key)
                data[key] = val.func

        keys = []
        parameters = [
            Parameter("self", kind=Parameter.POSITIONAL_ONLY)
        ]
        defaults = []

        if "__annotations__" in data:
            annotations = data.pop("__annotations__")
            for key, dtype in annotations.items():
                default_val = None
                if key in data:
                    default_val = data.pop(key)

                data[key] = Accessor(key, default=default_val, validators=validators.get(key))
                keys.append(key)
                parameters.append(Parameter(key, Parameter.KEYWORD_ONLY, default=default_val, annotation=dtype))
                defaults.append(default_val)
                                        
        def __init__(self, *args, **kwargs):
            for key, val in kwargs.items():
                if key in keys:
                    setattr(self, key, val)
            self.build(*args, **kwargs)

        def __copy__(self):
            cls = self.__class__

            result = cls.__new__(cls)
            result.__dict__.update(self.__dict__)
    
            for key in keys:
                setattr(result, key, getattr(self, key))
    
            return result
    
        def __deepcopy__(self, memo: Dict[int, Any]):
            cls = self.__class__
            result = cls.__new__(cls)
            memo[id(self)] = result

            for k, v in self.__dict__.items():
                setattr(result, k, deepcopy(v, memo))
                
            for key in keys:
                val = getattr(self, key)
                setattr(result, key, deepcopy(val))

            return result
              
        setattr(
            __init__,
            "__signature__",
            Signature(parameters=parameters)
        )
        __init__.__defaults__ = tuple(defaults)


        data["__init__"] = __init__ 
        data["__copy__"] = __copy__
        data["__deepcopy__"] = __deepcopy__
        
        return super().__new__(cls, name, bases, data)            
    
    
class Dataclass(metaclass=DataMetaclass):
    def build(self, *args, **kwargs):
        pass


class Field:
    def __init__(self, default: Any=Ellipsis, /, alias: str = None, default_factory: Callable = None):
        self.default = default
        self.alias = alias
        self.default_factory = default_factory

In [57]:
class Foo(Dataclass):
    age: int = 18
    name: Dict[str, Any] = {}
        
    @validator("age")
    def is_adult(self, age: int):
        return age > 18
    
    @validator("age")
    def is_adult(self, age: float):
        return age > 18

In [58]:
c = Foo()
d = Foo()

In [59]:
inspect.signature(Foo)

<Signature (*, age: int = 18, name: Dict[str, Any] = {})>

In [145]:
d.name

{}

In [146]:
b = c

In [147]:
b.name

{'name': 'datnh21'}

In [148]:
b = deepcopy(c)

In [150]:
b.name['name'] = "a"

In [152]:
b.name

{'name': 'a'}

In [60]:
a = list(a + 1 for a in range(10))

In [61]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]